Este notebook tem como objetivo aplicar testes estatísticos formais para validar hipóteses levantadas durante a análise exploratória. Diferentemente do EDA, aqui as conclusões são baseadas em inferência estatística, com hipóteses nulas bem definidas, testes apropriados e interpretação cuidadosa dos resultados.

Os resultados obtidos neste notebook orientam decisões de modelagem e ajudam a justificar escolhas feitas nos modelos de regressão linear e modelos mais complexos.

### Importações e Configurações

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

### Carregamento dos Dados com Engenharia de Atributos

In [ ]:
df = pd.read_csv('../3_feature_engineering/outputs/data/df_features.csv', parse_dates=['Date'], delimiter=',')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,week,week_sin,week_cos,is_holiday,holiday_week,rolling_mean_4,lag_1,lag_52,Type_B,Type_C
0,1,1,2011-02-04,21665.76,False,42.27,2.989,0.0,0.0,0.0,...,5,0.568065,0.822984,0,0,18707.0275,18461.18,24924.50,False,False
1,1,1,2011-02-11,37887.17,True,36.39,3.022,0.0,0.0,0.0,...,6,0.663123,0.748511,1,6,23838.8950,21665.76,46039.49,False,False
2,1,1,2011-02-18,46845.87,False,57.36,3.045,0.0,0.0,0.0,...,7,0.748511,0.663123,0,0,31214.9950,37887.17,41595.55,False,False
3,1,1,2011-02-25,19363.83,False,62.90,3.065,0.0,0.0,0.0,...,8,0.822984,0.568065,0,0,31440.6575,46845.87,19403.54,False,False
4,1,1,2011-03-04,20327.61,False,59.58,3.288,0.0,0.0,0.0,...,9,0.885456,0.464723,0,0,31106.1200,19363.83,21827.90,False,False


Os testes estatísticos são realizados sobre o dataset já limpo e com engenharia de atributos básica aplicada, garantindo que os resultados não sejam influenciados por inconsistências previamente identificadas.

### Definição Base para os Testes

In [4]:
ALPHA = 0.05

O nível de significância adotado é de 5%, valor padrão em aplicações estatísticas, equilibrando risco de falsos positivos e poder do teste.

### Normalidade

In [5]:
# Teste de Normalidade (Weekly_Sales)

sample = df['Weekly_Sales'].sample(5000, random_state=42)
stat, p_value = stats.shapiro(sample)

stat, p_value

(np.float64(0.6953281416154348), np.float64(8.363168870595714e-70))

O teste de Shapiro-Wilk rejeita fortemente a hipótese de normalidade para a variável Weekly_Sales. Esse resultado confirma a assimetria e caudas pesadas observadas durante o EDA, indicando que a suposição de normalidade dos resíduos não pode ser assumida sem transformações adicionais.

In [6]:
# Normalidade Após Transformação log

sample_log = df['Weekly_Sales_log'].sample(5000, random_state=42)
stats.shapiro(sample_log)

ShapiroResult(statistic=np.float64(0.9272802834030068), pvalue=np.float64(5.314685033159938e-44))

Embora a transformação logarítmica melhore substancialmente a forma da distribuição, o teste de Shapiro-Wilk ainda rejeita a hipótese de normalidade. Isso sugere que, apesar da transformação reduzir assimetrias extremas, a distribuição permanece distante de uma normal ideal, o que reforça a necessidade de diagnósticos adicionais nos resíduos do modelo.

### Testes de Diferença de Médias

**Hipótese: feriados alteram as vendas?**

Hipóteses:

* H₀: média das vendas é igual em semanas com e sem feriado
* H₁: médias são diferentes

In [7]:
holiday = df[df['is_holiday'] == 1]['Weekly_Sales']
non_holiday = df[df['is_holiday'] == 0]['Weekly_Sales']

stats.ttest_ind(holiday, non_holiday, equal_var=False)

TtestResult(statistic=np.float64(6.595975746062489), pvalue=np.float64(4.3368325010149846e-11), df=np.float64(18854.548729267284))

O teste t indica uma diferença estatisticamente significativa entre as médias de vendas em semanas com e sem feriado. Esse resultado confirma que o efeito de feriados não é aleatório, embora a magnitude desse impacto precise ser avaliada separadamente por meio de métricas de tamanho de efeito.

In [8]:
# Tamanho do Efeito

def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(((nx - 1) * x.var() + (ny - 1) * y.var()) / (nx + ny - 2))
    return (x.mean() - y.mean()) / pooled_std

cohens_d(holiday, non_holiday)

np.float64(0.06128951324319936)

Apesar da significância estatística, o tamanho do efeito é pequeno, indicando que feriados, isoladamente, explicam apenas uma fração limitada da variabilidade nas vendas. Esse contraste entre significância estatística e efeito prático sugere que o impacto dos feriados deve ser interpretado com cautela em modelos preditivos.

### Homocedasticidade

In [9]:
# Teste de Levene

stats.levene(holiday, non_holiday)

LeveneResult(statistic=np.float64(60.98634361631407), pvalue=np.float64(5.768441316500476e-15))

O teste de Levene sugere variâncias diferentes entre semanas com e sem feriado, indicando heterocedasticidade. Esse resultado aponta para possíveis limitações da regressão linear clássica e reforça a necessidade de diagnóstico de resíduos.

### Associação entre Variáveis Contínuas

In [10]:
# Correlaçãao de Pearson (linear)

stats.pearsonr(df['CPI'], df['Weekly_Sales'])

PearsonRResult(statistic=np.float64(-0.019745708512027567), pvalue=np.float64(7.06879365776079e-24))

A correlação linear entre CPI e vendas é fraca, sugerindo que o efeito dessa variável pode ser indireto ou não linear, o que limita sua contribuição em modelos lineares simples.

In [11]:
# Correlação de Spearman (monotônica)

stats.spearmanr(df['CPI'], df['Weekly_Sales'])

SignificanceResult(statistic=np.float64(-0.026589567089869835), pvalue=np.float64(6.10359528284056e-42))

A correlação de Spearman também é baixa, indicando ausência de relação monotônica forte entre CPI e vendas, o que ajuda a explicar o baixo poder explicativo dessa variável isoladamente.

### Testes de Independência (Chi-Quadrado)

In [12]:
contigency = pd.crosstab(df['Type_B'], df['is_holiday'])
stats.chi2_contingency(contigency)

Chi2ContingencyResult(statistic=np.float64(0.013771461807958454), pvalue=np.float64(0.9065812212275112), dof=1, expected_freq=array([[148902.73785699,  10504.26214301],
       [ 94277.26214301,   6650.73785699]]))

O teste qui-quadrado avalia a independência entre tipo de loja e ocorrência de feriados. A ausência de associação forte sugere que o impacto de feriados é relativamente homogêneo entre os tipos de loja.

### Estimação e Intervalos de Confiança

In [13]:
# Intervalo de Confiança da Média

mean = df['Weekly_Sales'].mean()
sem = stats.sem(df['Weekly_Sales'])
ci = stats.t.interval(0.95, len(df) - 1, loc = mean, scale = sem)

mean, ci

(np.float64(16440.891232834616),
 (np.float64(16353.334225574994), np.float64(16528.44824009424)))

O intervalo de confiança fornece uma estimativa da incerteza associada à média das vendas semanais, reforçando a importância de considerar variabilidade e não apenas valores pontuais.O intervalo de confiança relativamente estreito reflete o grande volume de observações disponíveis, fornecendo uma estimativa precisa da média global. No entanto, essa precisão não elimina a elevada variabilidade observada entre lojas, departamentos e períodos, o que reforça a necessidade de modelagem explicativa.